In [ ]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
import numpy as np
import subprocess
from scipy.stats import variation
from time import time
from collections import defaultdict
from line_plots import plot_line

from scale_to_latex import get_columnwidth, get_textwidth, get_figsize

In [ ]:
# Folder where nodeinfos are stored
nif = "node_info/"

# Folder where results are stored
rf = "results/"

algorithms = [
    "line",
    "node2vec",
    "sdne",
    "hope",
    "graphsage",
]

datasets = [
    "facebook",
    "protein",
    "blogcatalog",
    "wikipedia",
    "cora"
]

experiments = [
    "aligned_cossim",
    "20nn_overlap",
    "20nn_jaccard",
    "20nn_ranks",
    "20nn_angdiv",
    "rns",
    "linproc_cossim",   
]

windows = [25, 5, 25]
replace_dict = {"line":"LINE", "hope":"HOPE", "sdne":"SDNE", "graphsage":"GraphSAGE", "node2vec":"node2vec",
                "aligned_cossim":"Aligned Cosine Similarity", "20nn_overlap": "20-NN Overlap", 
                "20nn_jaccard": "20-NN Jaccard Similarity", "20nn_ranks": "20-NN Ranking Invariance",
                "facebook":"Facebook", "protein":"Protein", "blogcatalog":"BlogCatalog", "wikipedia": "Wikipedia", 
                "cora": "Cora", "page_rank":"PageRank", "coreness":"Coreness", "closeness":"Closeness",
                "rns": "Ranked Neighborhood Stability", "linproc_cossim":"LinProc",
                "20nn_angdiv" : "20-NN Angle Divergence"}
color_dict = {"LINE":"#E30066", "HOPE":"#612158", "SDNE":"#F6A800", "GraphSAGE":"#00549F", "node2vec":"#57AB27"}

In [ ]:
"""Compute a DataFrame that can be used for plotting"""
start_time = time()
df_dict = defaultdict(list)
df = pd.DataFrame()
for experiment in experiments:
    for dataset in datasets:
        print(experiment, dataset)
        data = [rf + "_".join([algo, dataset, experiment]) + ".npy" for algo in algorithms]
        mode, k, emb_dir = None, None, None
        info_file = nif + dataset + ".node_info"
        xscale = "log"
        filter_type = "mean"  # kind of rolling filter over y values
        window_sizes = windows
        ylabel = "mean of means"
        ylims = [0]
        save_path = "tmp/" + dataset + experiment
        mf_frame = plot_line(data, mode, info_file, xscale, window_sizes, k, emb_dir, filter_type, ylabel, ylims, save_path)
        mf_frame.insert(0, 'dataset', dataset)
        mf_frame.insert(0, 'experiment', experiment)
        df = df.append(mf_frame)
            
            
print(f"Computing the data frame took {time()-start_time} seconds.")
df = df.replace(to_replace=replace_dict)

In [ ]:
# Closeness vs aligned cossim

columnwidth = get_columnwidth()
textwidth = get_textwidth()
light_gray = ".8"
dark_gray =".15"
sns.set(context="notebook", style="ticks", font_scale=1, 
        rc={"axes.edgecolor": light_gray, "xtick.color": dark_gray,
            "ytick.color": dark_gray, "xtick.bottom": True,
            "font.size":8,"axes.titlesize":6,"axes.labelsize":6, "xtick.labelsize":6, "legend.fontsize":6, 
            "ytick.labelsize":6, "axes.linewidth":1, 
            "xtick.minor.width":0.5, "xtick.major.width":0.5,
            "ytick.minor.width":0.5, "ytick.major.width":0.5, "lines.linewidth": 0.7,
            "xtick.major.size": 3,
            "ytick.major.size": 3,
            "xtick.minor.size": 2,
            "ytick.minor.size": 2,
           })

width, height, aspect = get_figsize(textwidth, wf=1/5)
print(width, height, aspect)
legend_yoffset = -0.5 
plot_df = df
plot_df["algorithm"] = pd.Categorical(plot_df["algorithm"], ["HOPE", "LINE", "node2vec", "SDNE", "GraphSAGE"])
plot_df = plot_df.sort_values(by=["dataset", "algorithm"])
display(plot_df.head())
plot_df.rename(columns={"algorithm":"Algorithm"}, inplace=True)

g = sns.FacetGrid(data=plot_df.loc[(plot_df.experiment == "Aligned Cosine Similarity") & (plot_df.statistic == "Closeness")],
                     hue="Algorithm", col="dataset", palette=color_dict, sharex=False, height=1*width, aspect=1)
g.map(plt.plot, "x", "y")
g.axes[0,3].legend(loc='upper center', bbox_to_anchor=(-0.8, legend_yoffset), fancybox=False, shadow=False, ncol=5)
g.set_ylabels(f"Mean of Mean\nAligned Cosine Similarity")
g.set_titles("{col_name}")
g.set_xlabels("Closeness")
for ax in g.axes[0, :]:
    ax.set_xscale("log")
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    ax.set_yticks(np.arange(0, 1.1, 0.2))
     
g.axes[0,0].xaxis.set_minor_formatter(FormatStrFormatter('%.1f'))
g.axes[0,2].xaxis.set_minor_formatter(FormatStrFormatter('%.1f'))
g.axes[0,3].set_xlim(right=0.55)
g.axes[0,3].xaxis.set_minor_formatter(FormatStrFormatter('%.1f'))
g.axes[0,4].set_xlim(right=0.55)
g.axes[0,4].set_xticks([0.45,0.5,0.55])
g.axes[0,4].xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
g.axes[0,4].minorticks_off()

g.savefig("plots/cossim_closeness_all.pdf")

In [ ]:
# Closeness vs 20-NN Jaccard Similarity

columnwidth = get_columnwidth()
textwidth = get_textwidth()
light_gray = ".8"
dark_gray =".15"
sns.set(context="notebook", style="ticks", font_scale=1, 
        rc={"axes.edgecolor": light_gray, "xtick.color": dark_gray,
            "ytick.color": dark_gray, "xtick.bottom": True,
            "font.size":8,"axes.titlesize":6,"axes.labelsize":6, "xtick.labelsize":6, "legend.fontsize":6, 
            "ytick.labelsize":6, "axes.linewidth":1, 
            "xtick.minor.width":0.5, "xtick.major.width":0.5,
            "ytick.minor.width":0.5, "ytick.major.width":0.5, "lines.linewidth": 0.7,
            "xtick.major.size": 3,
            "ytick.major.size": 3,
            "xtick.minor.size": 2,
            "ytick.minor.size": 2,
           })

width, height, aspect = get_figsize(textwidth, wf=1/5)
print(width, height, aspect)
legend_yoffset = -0.5 
plot_df = df
plot_df["algorithm"] = pd.Categorical(plot_df["algorithm"], ["HOPE", "LINE", "node2vec", "SDNE", "GraphSAGE"])
plot_df = plot_df.sort_values(by=["dataset", "algorithm"])
display(plot_df.head())
plot_df.rename(columns={"algorithm":"Algorithm"}, inplace=True)

g = sns.FacetGrid(data=plot_df.loc[(plot_df.experiment == "20-NN Jaccard Similarity") & (plot_df.statistic == "Closeness")],
                     hue="Algorithm", col="dataset", palette=color_dict, sharex=False, height=1*width, aspect=1)
g.map(plt.plot, "x", "y")
g.axes[0,3].legend(loc='upper center', bbox_to_anchor=(-0.8, legend_yoffset), fancybox=False, shadow=False, ncol=5)
g.set_ylabels(f"Mean of Mean\n20-NN Jaccard Similarity")
g.set_titles("{col_name}")
g.set_xlabels("Closeness")
for ax in g.axes[0, :]:
    ax.set_xscale("log")
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    ax.set_yticks(np.arange(0, 1.1, 0.2))
    
g.axes[0,0].xaxis.set_minor_formatter(FormatStrFormatter('%.1f'))
g.axes[0,2].xaxis.set_minor_formatter(FormatStrFormatter('%.1f'))
g.axes[0,3].set_xlim(right=0.55)
g.axes[0,3].xaxis.set_minor_formatter(FormatStrFormatter('%.1f'))
g.axes[0,4].set_xlim(right=0.55)
g.axes[0,4].set_xticks([0.45,0.5,0.55])
g.axes[0,4].xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
g.axes[0,4].minorticks_off()
g.savefig("plots/jaccard_closeness_all.pdf")

In [ ]:
# Closeness vs 20-NN Ranking Invariance

columnwidth = get_columnwidth()
textwidth = get_textwidth()
light_gray = ".8"
dark_gray =".15"
sns.set(context="notebook", style="ticks", font_scale=1, 
        rc={"axes.edgecolor": light_gray, "xtick.color": dark_gray,
            "ytick.color": dark_gray, "xtick.bottom": True,
            "font.size":8,"axes.titlesize":6,"axes.labelsize":6, "xtick.labelsize":6, "legend.fontsize":6, 
            "ytick.labelsize":6, "axes.linewidth":1, 
            "xtick.minor.width":0.5, "xtick.major.width":0.5,
            "ytick.minor.width":0.5, "ytick.major.width":0.5, "lines.linewidth": 0.7,
            "xtick.major.size": 3,
            "ytick.major.size": 3,
            "xtick.minor.size": 2,
            "ytick.minor.size": 2,
           })

width, height, aspect = get_figsize(textwidth, wf=1/5)
print(width, height, aspect)
legend_yoffset = -0.5 
plot_df = df
plot_df["algorithm"] = pd.Categorical(plot_df["algorithm"], ["HOPE", "LINE", "node2vec", "SDNE", "GraphSAGE"])
plot_df = plot_df.sort_values(by=["dataset", "algorithm"])
display(plot_df.head())
plot_df.rename(columns={"algorithm":"Algorithm"}, inplace=True)

g = sns.FacetGrid(data=plot_df.loc[(plot_df.experiment == "20-NN Ranking Invariance") & (plot_df.statistic == "Closeness")],
                     hue="Algorithm", col="dataset", palette=color_dict, sharex=False, height=1*width, aspect=1)
g.map(plt.plot, "x", "y")
g.axes[0,3].legend(loc='upper center', bbox_to_anchor=(-0.8, legend_yoffset), fancybox=False, shadow=False, ncol=5)
g.set_ylabels(f"Mean of Mean\n20-NN Ranking Invariance")
#plt.legend(loc="best")
g.set_titles("{col_name}")
g.set_xlabels("Closeness")
for ax in g.axes[0, :]:
    ax.set_xscale("log")
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    ax.set_yticks(np.arange(0, 1.1, 0.2))
    
g.axes[0,0].xaxis.set_minor_formatter(FormatStrFormatter('%.1f'))
g.axes[0,2].xaxis.set_minor_formatter(FormatStrFormatter('%.1f'))
g.axes[0,3].set_xlim(right=0.55)
g.axes[0,3].xaxis.set_minor_formatter(FormatStrFormatter('%.1f'))
g.axes[0,4].set_xlim(right=0.55)
g.axes[0,4].set_xticks([0.45,0.5,0.55])
g.axes[0,4].xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
g.axes[0,4].minorticks_off()
g.savefig("plots/ranks_closeness_all.pdf")

In [ ]:
# Closeness vs Ranked Neighborhood Stability

columnwidth = get_columnwidth()
textwidth = get_textwidth()
light_gray = ".8"
dark_gray =".15"
sns.set(context="notebook", style="ticks", font_scale=1, 
        rc={"axes.edgecolor": light_gray, "xtick.color": dark_gray,
            "ytick.color": dark_gray, "xtick.bottom": True,
            "font.size":8,"axes.titlesize":6,"axes.labelsize":6, "xtick.labelsize":6, "legend.fontsize":6, 
            "ytick.labelsize":6, "axes.linewidth":1, 
            "xtick.minor.width":0.5, "xtick.major.width":0.5,
            "ytick.minor.width":0.5, "ytick.major.width":0.5, "lines.linewidth": 0.7,
            "xtick.major.size": 3,
            "ytick.major.size": 3,
            "xtick.minor.size": 2,
            "ytick.minor.size": 2,
           })

width, height, aspect = get_figsize(textwidth, wf=1/5)
print(width, height, aspect)
legend_yoffset = -0.5 
plot_df = df
plot_df["algorithm"] = pd.Categorical(plot_df["algorithm"], ["HOPE", "LINE", "node2vec", "SDNE", "GraphSAGE"])
plot_df = plot_df.sort_values(by=["dataset", "algorithm"])
display(plot_df.head())
plot_df.rename(columns={"algorithm":"Algorithm"}, inplace=True)

g = sns.FacetGrid(data=plot_df.loc[(plot_df.experiment == "Ranked Neighborhood Stability") & (plot_df.statistic == "Closeness")],
                     hue="Algorithm", col="dataset", palette=color_dict, sharex=False, height=1*width, aspect=1)
g.map(plt.plot, "x", "y")

g.axes[0,3].legend(loc='upper center', bbox_to_anchor=(-0.8, legend_yoffset), fancybox=False, shadow=False, ncol=5)
g.set_ylabels(f"Mean of Mean\nRanked Neighborhood Stability")
g.set_titles("{col_name}")
g.set_xlabels("Closeness")
for ax in g.axes[0, :]:
    ax.set_xscale("log")
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    ax.set_yticks(np.arange(0, 1.1, 0.2))
    
g.axes[0,0].xaxis.set_minor_formatter(FormatStrFormatter('%.1f'))
g.axes[0,2].xaxis.set_minor_formatter(FormatStrFormatter('%.1f'))
g.axes[0,3].set_xlim(right=0.55)
g.axes[0,3].xaxis.set_minor_formatter(FormatStrFormatter('%.1f'))
g.axes[0,4].set_xlim(right=0.55)
g.axes[0,4].set_xticks([0.45,0.5,0.55])
g.axes[0,4].xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
g.axes[0,4].minorticks_off()
g.savefig("plots/rns_closeness_all.pdf")

In [ ]:
# Closeness vs 20-NN Angle Divergence

columnwidth = get_columnwidth()
textwidth = get_textwidth()
light_gray = ".8"
dark_gray =".15"
sns.set(context="notebook", style="ticks", font_scale=1, 
        rc={"axes.edgecolor": light_gray, "xtick.color": dark_gray,
            "ytick.color": dark_gray, "xtick.bottom": True,
            "font.size":8,"axes.titlesize":6,"axes.labelsize":6, "xtick.labelsize":6, "legend.fontsize":6, 
            "ytick.labelsize":6, "axes.linewidth":1, 
            "xtick.minor.width":0.5, "xtick.major.width":0.5,
            "ytick.minor.width":0.5, "ytick.major.width":0.5, "lines.linewidth": 0.7,
            "xtick.major.size": 3,
            "ytick.major.size": 3,
            "xtick.minor.size": 2,
            "ytick.minor.size": 2,
           })

width, height, aspect = get_figsize(textwidth, wf=1/5)
print(width, height, aspect)
legend_yoffset = -0.5 
plot_df = df
plot_df["algorithm"] = pd.Categorical(plot_df["algorithm"], ["HOPE", "LINE", "node2vec", "SDNE", "GraphSAGE"])
plot_df = plot_df.sort_values(by=["dataset", "algorithm"])
display(plot_df.head())
plot_df.rename(columns={"algorithm":"Algorithm"}, inplace=True)

g = sns.FacetGrid(data=plot_df.loc[(plot_df.experiment == "20-NN Angle Divergence") & (plot_df.statistic == "Closeness")],
                     hue="Algorithm", col="dataset", palette=color_dict, sharex=False, height=1*width, aspect=1)
g.map(plt.plot, "x", "y")

g.set_ylabels(f"Mean of Mean\n 20-NN Angle Divergence")
g.set_titles("{col_name}")
g.set_xlabels("Closeness")
for ax in g.axes[0, :]:
    ax.set_xscale("log")
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    ax.set_yticks([0,10,20,30])
    
g.axes[0,0].xaxis.set_minor_formatter(FormatStrFormatter('%.1f'))
g.axes[0,2].xaxis.set_minor_formatter(FormatStrFormatter('%.1f'))
g.axes[0,3].set_xlim(right=0.55)
g.axes[0,3].xaxis.set_minor_formatter(FormatStrFormatter('%.1f'))
g.axes[0,4].set_xlim(right=0.55)
g.axes[0,4].set_xticks([0.45,0.5,0.55])
g.axes[0,4].xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
g.axes[0,4].minorticks_off()

g.axes[0,2].legend(loc='upper center', bbox_to_anchor=(0.37, legend_yoffset), fancybox=False, shadow=False, ncol=5)

g.savefig("plots/angdiv_closeness_all.pdf")

In [ ]:
# PageRank vs Aligned Cosine Similarity

columnwidth = get_columnwidth()
textwidth = get_textwidth()
light_gray = ".8"
dark_gray =".15"
sns.set(context="notebook", style="ticks", font_scale=1, 
        rc={"axes.edgecolor": light_gray, "xtick.color": dark_gray,
            "ytick.color": dark_gray, "xtick.bottom": True,
            "font.size":8,"axes.titlesize":6,"axes.labelsize":6, "xtick.labelsize":6, "legend.fontsize":6, 
            "ytick.labelsize":6, "axes.linewidth":1, 
            "xtick.minor.width":0.5, "xtick.major.width":0.5,
            "ytick.minor.width":0.5, "ytick.major.width":0.5, "lines.linewidth": 0.7,
            "xtick.major.size": 3,
            "ytick.major.size": 3,
            "xtick.minor.size": 2,
            "ytick.minor.size": 2,
           })

width, height, aspect = get_figsize(textwidth, wf=1/5)
print(width, height, aspect)
legend_yoffset = -0.5 
plot_df = df
plot_df["algorithm"] = pd.Categorical(plot_df["algorithm"], ["HOPE", "LINE", "node2vec", "SDNE", "GraphSAGE"])
plot_df = plot_df.sort_values(by=["dataset", "algorithm"])
display(plot_df.head())
plot_df.rename(columns={"algorithm":"Algorithm"}, inplace=True)

g = sns.FacetGrid(data=plot_df.loc[(plot_df.experiment == "Aligned Cosine Similarity") & (plot_df.statistic == "PageRank")],
                     hue="Algorithm", col="dataset", palette=color_dict, sharex=False, height=1*width, aspect=1)
g.map(plt.plot, "x", "y")
g.axes[0,3].legend(loc='upper center', bbox_to_anchor=(-0.8, legend_yoffset), fancybox=False, shadow=False, ncol=5)
g.set_ylabels(f"Mean of Mean\nAligned Cosine Similarity")
g.set_titles("{col_name}")
g.set_xlabels("PageRank")
for ax in g.axes[0, :]:
    ax.set_xscale("log")
    ax.set_yticks(np.arange(0, 1.1, 0.2))
    
g.savefig("plots/cossim_pagerank_all.pdf")

In [ ]:
# PageRank vs 20-NN Jaccard Similarity

columnwidth = get_columnwidth()
textwidth = get_textwidth()
light_gray = ".8"
dark_gray =".15"
sns.set(context="notebook", style="ticks", font_scale=1, 
        rc={"axes.edgecolor": light_gray, "xtick.color": dark_gray,
            "ytick.color": dark_gray, "xtick.bottom": True,
            "font.size":8,"axes.titlesize":6,"axes.labelsize":6, "xtick.labelsize":6, "legend.fontsize":6, 
            "ytick.labelsize":6, "axes.linewidth":1, 
            "xtick.minor.width":0.5, "xtick.major.width":0.5,
            "ytick.minor.width":0.5, "ytick.major.width":0.5, "lines.linewidth": 0.7,
            "xtick.major.size": 3,
            "ytick.major.size": 3,
            "xtick.minor.size": 2,
            "ytick.minor.size": 2,
           })

width, height, aspect = get_figsize(textwidth, wf=1/5)
print(width, height, aspect)
legend_yoffset = -0.5 
plot_df = df
plot_df["algorithm"] = pd.Categorical(plot_df["algorithm"], ["HOPE", "LINE", "node2vec", "SDNE", "GraphSAGE"])
plot_df = plot_df.sort_values(by=["dataset", "algorithm"])
display(plot_df.head())
plot_df.rename(columns={"algorithm":"Algorithm"}, inplace=True)

g = sns.FacetGrid(data=plot_df.loc[(plot_df.experiment == "20-NN Jaccard Similarity") & (plot_df.statistic == "PageRank")],
                     hue="Algorithm", col="dataset", palette=color_dict, sharex=False, height=1*width, aspect=1)
g.map(plt.plot, "x", "y")
g.axes[0,3].legend(loc='upper center', bbox_to_anchor=(-0.8, legend_yoffset), fancybox=False, shadow=False, ncol=5)
g.set_ylabels(f"Mean of Mean\n20-NN Jaccard Similarity")
g.set_titles("{col_name}")
g.set_xlabels("PageRank")
for ax in g.axes[0, :]:
    ax.set_xscale("log")
    ax.set_yticks(np.arange(0, 1.1, 0.2))
    
g.savefig("plots/jaccard_pagerank_all.pdf")

In [ ]:
# PageRank vs 20-NN Ranking Invariance

columnwidth = get_columnwidth()
textwidth = get_textwidth()
light_gray = ".8"
dark_gray =".15"
sns.set(context="notebook", style="ticks", font_scale=1, 
        rc={"axes.edgecolor": light_gray, "xtick.color": dark_gray,
            "ytick.color": dark_gray, "xtick.bottom": True,
            "font.size":8,"axes.titlesize":6,"axes.labelsize":6, "xtick.labelsize":6, "legend.fontsize":6, 
            "ytick.labelsize":6, "axes.linewidth":1, 
            "xtick.minor.width":0.5, "xtick.major.width":0.5,
            "ytick.minor.width":0.5, "ytick.major.width":0.5, "lines.linewidth": 0.7,
            "xtick.major.size": 3,
            "ytick.major.size": 3,
            "xtick.minor.size": 2,
            "ytick.minor.size": 2,
           })

width, height, aspect = get_figsize(textwidth, wf=1/5)
print(width, height, aspect)
legend_yoffset = -0.5 
plot_df = df
plot_df["algorithm"] = pd.Categorical(plot_df["algorithm"], ["HOPE", "LINE", "node2vec", "SDNE", "GraphSAGE"])
plot_df = plot_df.sort_values(by=["dataset", "algorithm"])
display(plot_df.head())
plot_df.rename(columns={"algorithm":"Algorithm"}, inplace=True)

g = sns.FacetGrid(data=plot_df.loc[(plot_df.experiment == "20-NN Ranking Invariance") & (plot_df.statistic == "PageRank")],
                     hue="Algorithm", col="dataset", palette=color_dict, sharex=False, height=1*width, aspect=1)
g.map(plt.plot, "x", "y")
g.axes[0,3].legend(loc='upper center', bbox_to_anchor=(-0.8, legend_yoffset), fancybox=False, shadow=False, ncol=5)
g.set_ylabels(f"Mean of Mean\n20-NN Ranking Invariance")
g.set_titles("{col_name}")
g.set_xlabels("PageRank")
for ax in g.axes[0, :]:
    ax.set_xscale("log")
    ax.set_yticks(np.arange(0, 1.1, 0.2))
    
g.savefig("plots/ranks_pagerank_all.pdf")

In [ ]:
# PageRank vs Ranked Neighborhood Stability

columnwidth = get_columnwidth()
textwidth = get_textwidth()
light_gray = ".8"
dark_gray =".15"
sns.set(context="notebook", style="ticks", font_scale=1, 
        rc={"axes.edgecolor": light_gray, "xtick.color": dark_gray,
            "ytick.color": dark_gray, "xtick.bottom": True,
            "font.size":8,"axes.titlesize":6,"axes.labelsize":6, "xtick.labelsize":6, "legend.fontsize":6, 
            "ytick.labelsize":6, "axes.linewidth":1, 
            "xtick.minor.width":0.5, "xtick.major.width":0.5,
            "ytick.minor.width":0.5, "ytick.major.width":0.5, "lines.linewidth": 0.7,
            "xtick.major.size": 3,
            "ytick.major.size": 3,
            "xtick.minor.size": 2,
            "ytick.minor.size": 2,
           })

width, height, aspect = get_figsize(textwidth, wf=1/5)
print(width, height, aspect)
legend_yoffset = -0.5 
plot_df = df
plot_df["algorithm"] = pd.Categorical(plot_df["algorithm"], ["HOPE", "LINE", "node2vec", "SDNE", "GraphSAGE"])
plot_df = plot_df.sort_values(by=["dataset", "algorithm"])
display(plot_df.head())
plot_df.rename(columns={"algorithm":"Algorithm"}, inplace=True)

g = sns.FacetGrid(data=plot_df.loc[(plot_df.experiment == "Ranked Neighborhood Stability") & (plot_df.statistic == "PageRank")],
                     hue="Algorithm", col="dataset", palette=color_dict, sharex=False, height=1*width, aspect=1)
g.map(plt.plot, "x", "y")
g.axes[0,3].legend(loc='upper center', bbox_to_anchor=(-0.8, legend_yoffset), fancybox=False, shadow=False, ncol=5)
g.set_ylabels(f"Mean of Mean\nRanked Neighborhood Stability")
g.set_titles("{col_name}")
g.set_xlabels("PageRank")
for ax in g.axes[0, :]:
    ax.set_xscale("log")
    ax.set_yticks(np.arange(0, 1.1, 0.2))
    
g.savefig("plots/rns_pagerank_all.pdf")

In [ ]:
# PageRank vs 20-NN Angle Divergence

columnwidth = get_columnwidth()
textwidth = get_textwidth()
light_gray = ".8"
dark_gray =".15"
sns.set(context="notebook", style="ticks", font_scale=1, 
        rc={"axes.edgecolor": light_gray, "xtick.color": dark_gray,
            "ytick.color": dark_gray, "xtick.bottom": True,
            "font.size":8,"axes.titlesize":6,"axes.labelsize":6, "xtick.labelsize":6, "legend.fontsize":6, 
            "ytick.labelsize":6, "axes.linewidth":1, 
            "xtick.minor.width":0.5, "xtick.major.width":0.5,
            "ytick.minor.width":0.5, "ytick.major.width":0.5, "lines.linewidth": 0.7,
            "xtick.major.size": 3,
            "ytick.major.size": 3,
            "xtick.minor.size": 2,
            "ytick.minor.size": 2,
           })

width, height, aspect = get_figsize(textwidth, wf=1/5)
print(width, height, aspect)
legend_yoffset = -0.5 
plot_df = df
plot_df["algorithm"] = pd.Categorical(plot_df["algorithm"], ["HOPE", "LINE", "node2vec", "SDNE", "GraphSAGE"])
plot_df = plot_df.sort_values(by=["dataset", "algorithm"])
display(plot_df.head())
plot_df.rename(columns={"algorithm":"Algorithm"}, inplace=True)

g = sns.FacetGrid(data=plot_df.loc[(plot_df.experiment == "20-NN Angle Divergence") & (plot_df.statistic == "PageRank")],
                     hue="Algorithm", col="dataset", palette=color_dict, sharex=False, height=1*width, aspect=1)
g.map(plt.plot, "x", "y")
g.axes[0,3].legend(loc='upper center', bbox_to_anchor=(-0.8, legend_yoffset), fancybox=False, shadow=False, ncol=5)
g.set_ylabels(f"Mean of Mean\n20-NN Angle Divergence")
g.set_titles("{col_name}")
g.set_xlabels("PageRank")
for ax in g.axes[0, :]:
    ax.set_xscale("log")
    ax.set_yticks([0,10,20,30])
    
g.savefig("plots/angdiv_pagerank_all.pdf")